In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/MyDrive/"
!unzip "/content/drive/MyDrive/dataset.zip"

ModuleNotFoundError: No module named 'google.colab'

In [31]:
%pip install pygad

import tensorflow as tf
import pygad as pg
import pygad.kerasga
import os
import numpy as np

imgDir = os.path.join('dataset', 'flowers')
imgSize = 32
batchSize = 128

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\rbfre\Github\CS420FinalProject\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [32]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(imgDir, image_size=(imgSize,imgSize), batch_size=batchSize)
testData = dataset.take(25)
trainData = dataset.skip(25)
yTrain = np.asarray([y for x,y in trainData.unbatch()])

Found 13838 files belonging to 14 classes.


In [33]:
# build verification model
inputs = tf.keras.layers.Input(shape=(imgSize, imgSize, 3))
x = tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), activation='relu')(inputs)
# x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(units=14, activation='softmax')(x)

# make model checkpoint to save weights
checkpoints = tf.keras.callbacks.ModelCheckpoint("verifyWeights", monitor="sparse_categorical_accuracy", save_best_only=True)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy', metrics='sparse_categorical_accuracy', optimizer='adam')
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 30, 30, 3)         84        
                                                                 
 flatten_4 (Flatten)         (None, 2700)              0         
                                                                 
 dropout_4 (Dropout)         (None, 2700)              0         
                                                                 
 dense_4 (Dense)             (None, 14)                37814     
                                                                 
Total params: 37,898
Trainable params: 37,898
Non-trainable params: 0
_________________________________________________________________


In [34]:

model.fit(trainData, epochs=5, callbacks=checkpoints)

Epoch 1/5
84/84 [==============================] - 5s 45ms/step - loss: 44.1633 - sparse_categorical_accuracy: 0.1056
Epoch 2/5
84/84 [==============================] - 5s 46ms/step - loss: 6.9604 - sparse_categorical_accuracy: 0.1153
Epoch 3/5
84/84 [==============================] - 5s 46ms/step - loss: 4.0119 - sparse_categorical_accuracy: 0.1212
Epoch 4/5
84/84 [==============================] - 5s 49ms/step - loss: 3.2364 - sparse_categorical_accuracy: 0.1243
Epoch 5/5
72/84 [========================>.....] - ETA: 0s - loss: 2.9278 - sparse_categorical_accuracy: 0.1273

KeyboardInterrupt: 

In [20]:
model.evaluate(testData)

25/25 [==============================] - 1s 41ms/step - loss: 3.0090 - sparse_categorical_accuracy: 0.1988


[3.0089855194091797, 0.19875000417232513]

In [13]:
print(yTrain.shape)

(10638,)


In [24]:
# make a kerasGA
myGA = pg.kerasga.KerasGA(model=model, num_solutions=20)

# this is our fitness func
def fitFunc(solution, solutionIndx):
    print('in fitness func')
    weightsMatrix = pg.kerasga.model_weights_as_matrix(model=model, weights_vector=solution)
    model.set_weights(weights=weightsMatrix)
    preds = model.predict(trainData)
    sparseCE = tf.keras.losses.SparseCategoricalCrossentropy()
    fitness = 1.0 / (sparseCE(yTrain, preds).numpy() + 0.00000001)
    return fitness

# progress callback for GA
def genCallback(gaInstance):
    print(f'Generation = {gaInstance.generations_completed}')
    print(f'fitness = {gaInstance.best_solution()[1]}')

In [25]:
# now make a GA instance with our variables
numGen = 5
numParents = 10
initPopulation = myGA.population_weights

gaInstance = pg.GA(num_generations=numGen,
                   num_parents_mating=numParents,
                   initial_population=initPopulation,
                   fitness_func=fitFunc,
                   on_generation=genCallback)

In [26]:
gaInstance.run()
gaInstance.plot_fitness(title="woO baby", linewidth=4)

in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
Generation = 1
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
in fitness func
fitness = 0.06777927801130573
in fitness func
in fitness func
in fitness func
in fitness func


KeyboardInterrupt: 